<a href="https://colab.research.google.com/github/patrycjaarcisz/learning-data-science/blob/main/SQLbasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# dataquest's SQL basics tutorial, analysing a dataset from the bike-sharing service Hubway

###the goal is to answer the following questions: 
What was the duration of the longest trip?
How many trips were taken by 'registered' users?
What was the average trip duration?
Do registered or casual users take longer trips?
Which bike was used for the most trips?
What is the average duration of trips by users over the age of 30?

source: https://www.dataquest.io/blog/sql-basics/

In [6]:
import sqlite3
import pandas as pd
db = sqlite3.connect('hubway.db')
def run_query(query):
  return pd.read_sql_query(query,db)

In [7]:
query = 'SELECT * FROM trips LIMIT 5;' # returns every column in trips table, limited to 5 rows
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981.0,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983.0,Female


In [8]:
query = 'SELECT duration, start_date FROM trips LIMIT 5;' # returns only duration and start_date columns
run_query(query)

,duration,start_date
0,9,2011-07-28 10:12:00
1,220,2011-07-28 10:21:00
2,56,2011-07-28 10:33:00
3,64,2011-07-28 10:35:00
4,12,2011-07-28 10:37:00


### What was the duration of the longest trip?

In [9]:
query = '''
SELECT duration FROM trips
ORDER BY duration DESC
LIMIT 1;
'''

# ASC ascending order, DESC descending order, by default ORDER BY sorts in ascending order

run_query(query)

,duration
0,9999


The longest trip lasted 9999 seconds, but we don't know whether this is really the longest trip or if the database was only set up to allow a four digit number.

In [11]:
query = '''
SELECT duration FROM trips
ORDER BY duration DESC
LIMIT 10;
'''

run_query(query)

,duration
0,9999
1,9998
2,9998
3,9997
4,9996
5,9996
6,9995
7,9995
8,9994
9,9994


What we see here is that there aren't a whole bunch of trips at 9999, so it doesn't look like we're cutting off the top end of our durations, but it's still difficult to tell whether that's the real length of the trip or just the maximum allowed value.

### How many trips were taken by 'registered' users?

In [13]:
# WHERE allows us to use a logical operator to specify which rows should be returned, for example:

query = '''
SELECT * FROM trips
WHERE duration > 9990;
'''

# query returns every column in trips table for each row with a duration longer than 9990 seconds

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,4768,9994,2011-08-03 17:16:00,22,2011-08-03 20:03:00,24,B00002,Casual,,,
1,8448,9991,2011-08-06 13:02:00,52,2011-08-06 15:48:00,24,B00174,Casual,,,
2,11341,9998,2011-08-09 10:42:00,40,2011-08-09 13:29:00,42,B00513,Casual,,,
3,24455,9995,2011-08-20 12:20:00,52,2011-08-20 15:07:00,17,B00552,Casual,,,
4,55771,9994,2011-09-14 15:44:00,40,2011-09-14 18:30:00,40,B00139,Casual,,,
5,81191,9993,2011-10-03 11:30:00,22,2011-10-03 14:16:00,36,B00474,Casual,,,
6,89335,9997,2011-10-09 02:30:00,60,2011-10-09 05:17:00,45,B00047,Casual,,,
7,124500,9992,2011-11-09 09:08:00,22,2011-11-09 11:55:00,40,B00387,Casual,,,
8,133967,9996,2011-11-19 13:48:00,4,2011-11-19 16:35:00,58,B00238,Casual,,,
9,147451,9996,2012-03-23 14:48:00,35,2012-03-23 17:35:00,33,B00550,Casual,,,


In [14]:
# WHERE can also combine multiple logical tests in our WHERE clause using AND or OR, foe example: we want to return the trips with a duration over 9990 seconds that also were taken by registered users:

query = '''
SELECT * FROM trips
WHERE (duration >= 9990) AND (sub_type = "Registered")
ORDER BY duration DESC;
'''

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,315737,9995,2012-07-03 18:28:00,12,2012-07-03 21:15:00,12,B00250,Registered,'02120,1964.0,Male


In [16]:
# COUNT(1) and COUNT(*) count up every row in our query

# a query that is going to return a total number of trips taken by registered users:

query = '''
SELECT COUNT(*) FROM trips
WHERE sub_type = "Registered";
'''

run_query(query)

,COUNT(*)
0,1105192


the column heading isn't particularly descriptive, if we want the results to be more readable we can use AS

In [17]:
query = '''
SELECT COUNT(*) AS "Total Trips by Registered Users"
FROM trips
WHERE sub_type = "Registered";
'''

run_query(query)

,Total Trips by Registered Users
0,1105192


### What was the average trip duration?

In [18]:
# SUM, AVG, MIN, MAX return sum, average, minimum and maximum respectively. 
# These along with COUNT, are known as aggregate functions.

In [19]:
query = '''
SELECT AVG(duration) AS "Average Duration"
FROM trips;
'''

run_query(query)

,Average Duration
0,912.409682


average trip duration is 912 seconds

### Do registered or casual users take longer trips?

In [20]:
# GROUP BY separates rows into groups based on the contents of a particular column
# and allows us to perform aggregate functions on each group

In [24]:
query = '''
SELECT sub_type, AVG(duration) AS "Average Duration"
FROM trips
GROUP BY sub_type;
'''

run_query(query)

,sub_type,Average Duration
0,Casual,1519.643897
1,Registered,657.026067


### Which bike was used for the most trips?

In [26]:
query = '''
SELECT bike_number as "Bike Number", COUNT(*) as "Number of Trips"
FROM trips
GROUP BY bike_number
ORDER BY COUNT(*) DESC
LIMIT 1;
'''

run_query(query)

,Bike Number,Number of Trips
0,B00490,2120


In [27]:
# we use GROUP BY to tell the COUNT function on line 1 of the query to count up each value for bike_number separately
# we have ORDER BY clause to sort the table in descending order and make sure our most-used bike is at the top
# we use LIMIT to restrict the output to the first row

### What is the average duration of trips by registered memebers over the age of 30?

In [28]:
query = '''
SELECT AVG(duration) AS "Average Duration by the Registered Members over 30"
FROM trips
WHERE (2021 - birth_date) > 30;
'''

run_query(query)

,Average Duration by the Registered Members over 30
0,915.167566


### Which station is the most frequent starting point?

In [29]:
# JOIN specifies which tables should be connected and ON specifies which columns in each table are related

In [30]:
query = '''
SELECT * FROM stations
LIMIT 5;
'''

run_query(query)

,id,station,municipality,lat,lng
0,3,Colleges of the Fenway,Boston,42.340021,-71.100812
1,4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
2,5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179
3,6,Cambridge St. at Joy St.,Boston,42.361284999999995,-71.06514
4,7,Fan Pier,Boston,42.353412,-71.044624


In [32]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips
INNER JOIN stations
ON trips.start_station = stations.id
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5; 
'''

run_query(query)

,Station,Count
0,South Station - 700 Atlantic Ave.,56123
1,Boston Public Library - 700 Boylston St.,41994
2,Charles Circle - Charles St. at Cambridge St.,35984
3,Beacon St / Mass Ave,35275
4,MIT at Mass Ave / Amherst St,33644


### Which stations are most frequently used for round trips?

In [34]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips
INNER JOIN stations
ON trips.start_station = stations.id
WHERE trips.start_station = trips.end_station
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''

run_query(query)

,Station,Count
0,The Esplanade - Beacon St. at Arlington St.,3064
1,Charles Circle - Charles St. at Cambridge St.,2739
2,Boston Public Library - 700 Boylston St.,2548
3,Boylston St. at Arlington St.,2163
4,Beacon St / Mass Ave,2144


### How many trips start and end in different municipalities?

In [36]:
query = '''
SELECT COUNT(trips.id) AS "Count"
FROM trips 
INNER JOIN stations AS start
ON trips.start_station = start.id
INNER JOIN stations AS end
ON trips.end_station = end.id
WHERE start.municipality != end.municipality;
'''

run_query(query)

,Count
0,309748


### How many trips incurred additional fees (lasted longer than 30 minutes)?

In [37]:
query = '''
SELECT * 
FROM trips
LIMIT 5; 
'''

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981.0,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983.0,Female


In [38]:
query = '''
SELECT COUNT(*) AS "Trips with Additional Fees"
FROM trips
WHERE duration > 1800;
'''

run_query(query)

,Trips with Additional Fees
0,123155


### Which bike was used for the longest total time?

In [40]:
query = '''
SELECT bike_number AS "Bike", SUM(duration) AS "Total Time"
FROM trips
GROUP BY bike_number
ORDER BY SUM(duration) DESC
LIMIT 1;
'''

run_query(query)

,Bike,Total Time
0,B00490,2058643


### Did registered or casual users take more round trips?

In [43]:
query = '''
SELECT sub_type AS "Type of User", COUNT(sub_type) AS "Number of Round Trips"
FROM trips INNER JOIN stations
ON trips.start_station = stations.id
WHERE trips.start_station = trips.end_station
GROUP BY sub_type
ORDER BY COUNT(sub_type) DESC
'''

run_query(query)


,Type of User,Number of Round Trips
0,Casual,41427
1,Registered,31635


Answer: Casual users took more round trips.